In [ ]:
# Create the translation mapping
french_to_english = {
    "id": "id",
    "titre": "title",
    "url": "source_url",
    "domaine": "domain",
    "prononciation": "date_pronounced",
    "intervenants": "speakers",
    "nom": "speaker_name",
    "qualite": "speaker_role", 
    "qualite_long": "speaker_role_extended",
    "auteur_moral": "corporate_author",
    "circonstance": "circumstance",
    "type_emetteur": "issuer_type",
    "type_document": "document_type",
    "thematiques": "themes",
    "descripteurs": "tags",
    "mise_en_ligne": "date_published",
    "mise_a_jour": "date_updated"
}

In [35]:
import json
import pandas as pd

with open("dataset/vp_discours.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [36]:
from datetime import datetime

def filter_by_date_range(data_list, begin_date_str, end_date_str):
    """
    Filters a list of dicts based on whether the 'prononciation' date 
    falls within the range [begin_date, end_date].
    """
    # Convert boundary strings to datetime objects once
    begin_date = datetime.strptime(begin_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    filtered_list = [
        entry for entry in data_list
        if begin_date <= datetime.strptime(entry["prononciation"], "%Y-%m-%d") <= end_date
    ]
    
    return filtered_list

In [38]:
begin_date_str = "2010-01-01"
end_date_str = "2023-12-31"

target_data = filter_by_date_range(data, begin_date_str, end_date_str)
print(f"Number of speeches between {begin_date_str} and {end_date_str}: {len(target_data)}")
print(json.dumps(target_data[0], ensure_ascii=False, indent=2))

Number of speeches between 2010-01-01 and 2023-12-31: 39124
{
  "id": "243000039",
  "titre": "Déclaration de Mme Élisabeth Borne, Première ministre, sur l'engagement des forces armées, Maripasoula (Guyane) le 31 décembre 2023.",
  "url": "https://www.vie-publique.fr/discours/292563-elisabeth-borne-31122023-l-engagement-des-formes-armees-et-voeux",
  "domaine": "Déclaration",
  "prononciation": "2023-12-31",
  "intervenants": [
    {
      "nom": "Élisabeth Borne",
      "qualite": "Première ministre",
      "qualite_long": null
    }
  ],
  "auteur_moral": [],
  "circonstance": null,
  "type_emetteur": "Premier ministre",
  "type_document": "Déclaration",
  "type_media": null,
  "media": null,
  "resume": null,
  "thematiques": [
    "Armée"
  ],
  "descripteurs": [
    "Guyane",
    "Voeux",
    "Militaire",
    "Police",
    "Gendarmerie"
  ],
  "mise_en_ligne": "2024-01-04",
  "mise_a_jour": "2025-05-13"
}


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_speech_script(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Check for errors
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Target the specific container shown in your image
        # We use the unique 'field--name-field-texte-integral' class
        container = soup.find('div', class_='field--name-field-texte-integral')
        
        if container:
            # Find all paragraphs within that container
            paragraphs = container.find_all('p')
            # Join the text of all paragraphs with a newline
            full_text = " </new_paragraph>".join([p.get_text(strip=True) for p in paragraphs])
        else:
            full_text = "Could not find the script container on this page."
        
        # This selects all <a> tags with class 'fr-tag' that are inside the 'vp-tags' div
        tags = []
        tag_elements = soup.select('div.vp-tags ul.vp-tags-list a.fr-tag')
        all_tags = [tag.get_text(strip=True) for tag in tag_elements]

        return {
            "text": full_text,
            "tag": all_tags
        }
    except Exception as e:
        return f"An error occurred: {e}"


https://www.vie-publique.fr/discours/301308-sebastien-lecornu-10122025-strategie-defense-nationale-devant-assemblee
{
  "text": "Madame la Présidente de l'Assemblée nationale,Mesdames et messieurs les députés,Monsieur le président de la Commission de la Défense nationale et des Forces armées, </new_paragraph>\"Les Français sont déjà trop portés à croire qu'ils peuvent dormir tranquilles, qu'ils n'ont qu'à s'en remettre à d'autres du soin de défendre leur indépendance. Il ne faut pas les encourager dans cette confiance naïve qu'ils paient ensuite par des ruines et des massacres. Il faut les encourager à compter sur eux-mêmes.\" Ces mots sont ceux du général de Gaulle confiés à son ministre Alain Peyrefitte. Ils traduisent une volonté partagée sur l'ensemble de ces bancs, celle de ne compter que sur nous-mêmes, de ne jamais dépendre des autres, d'être indépendants. C'est ce qui a justifié toute la construction de notre modèle d'armée depuis 1958 et qui justifie aujourd'hui notre réarmeme

In [ ]:
# Example usage
id = 123
print(data[id]['url'])
res = get_speech_script(data[id]['url'])
res['text'] = res['text'].replace('\n', ' ').replace('\xa0', ' ')
print(json.dumps(res, ensure_ascii=False, indent=2))

https://www.vie-publique.fr/discours/301123-stephanie-rist-26112025-centres-de-sante-pluriprofessionnels
{
  "text": "M. le présidentL'ordre du jour appelle le débat sur le thème : \"Les centres de santé pluriprofessionnels : antidote aux déserts médicaux ?\" </new_paragraph>Ce débat a été demandé par le groupe Écologiste et social. </new_paragraph>La conférence des présidents a décidé d'organiser ce débat en deux parties. Nous commencerons par une table ronde en présence de personnalités invitées d'une durée d'une heure ; puis, nous procéderons, après avoir entendu une intervention liminaire du gouvernement, à une séquence de questions-réponses d'une durée d'une heure également. La durée des questions et des réponses sera limitée à deux minutes, sans droit de réplique. </new_paragraph>Pour la première phase du débat, je souhaite la bienvenue à Mme Laure Artru, présidente de l'Association de citoyens contre les déserts médicaux, à M. Frédéric Villebrun, coprésident de l'Union syndicale